In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize,FreqDist
import string

In [3]:
from nltk.stem import PorterStemmer

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score
from sklearn.metrics import plot_confusion_matrix

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [7]:
import fasttext

In [8]:
import csv

In [9]:
from bs4 import BeautifulSoup

In [11]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    cleaned_text = cleaned_text.replace('\n',' ')
    return cleaned_text

# Example usage
html_text = "<p>This is <b>HTML</b> text with \n <a href='example.com'>tags</a>.</p>"
cleaned_text = remove_html_tags(html_text)
print(cleaned_text)

This is HTML text with   tags.


In [12]:
df = pd.read_csv(r"C:\Users\tanma\Downloads\train.csv")

In [13]:
df_test = pd.read_csv(r"C:\Users\tanma\Downloads\test.csv")
df_test

,problem,category
0,"<p>For graduating from middle school, Wayne r...",NaN
1,<p>Mrs. O'Hare bought `18` packs of pencils to...,NaN
2,<p>Annalise bought `p` pens last week and `15`...,NaN
3,<p>Felicia is decorating cupcakes for a party....,NaN
4,<p>Write an expression to match the phrase:</p...,NaN
...,...,...
120,<p>Veronica has `d` more dresses than Larisa d...,NaN
121,<p>Write an equation to match the phrase:</p><...,NaN
122,<p>Follow the directions below to create an eq...,NaN
123,<p>What is `7.5 ÷ 10`?</p><selectivedisplay da...,NaN


In [70]:
df_imp

,problem
0,"At a ring toss game at a carnival, Marco throw..."
1,What is `471 ÷ 100`? Interactive: Students fol...
2,Follow the directions below to create an equiv...
3,Solve the following system by substitution:\n`...
4,Solve the following system by substitution:\n`...
...,...
245,"Write an expression to match the phrase:""`9` p..."
246,"Identify the terms, coefficients, variables, a..."
247,Solve the following system by elimination:\n`-...
248,"Write an expression to match the phrase:""`8` t..."


In [14]:
stops = stopwords.words("english") + list(string.punctuation)

In [16]:
stops

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [10]:
# These stopwords would work as we do not have to remove expression x+y = 10 because that is important for our task

In [54]:
def process_questions(text):
    tokens = word_tokenize(text)
    up_tokens = [i.lower() for i in tokens if i.lower() not in stops]
    p = PorterStemmer()
    last = []
    for token in up_tokens:
        last.append(p.stem(token))
    return up_tokens

In [72]:
s = ""
for i in range(len(df_imp['problem'])):
    s = ""
    for word in df_imp['problem'].loc[i]:
        s+=word + " "
    df_imp['problem'].loc[i] = s

C:\Users\tanma\AppData\Local\Temp\ipykernel_36988\119967411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imp['problem'].loc[i] = s


In [73]:
x = df_imp['problem']
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=200)

In [74]:
X_train

109    identify terms coefficients variables constant...
242    juan li cyclist 's competing race juan covered...
233         solve following system substitution x 15 4x 
101    beekeeper ships box containing 30 containers h...
66     cynthia finished 15 math problems homework ton...
                             ...                        
42     determine number solutions following system eq...
68     determine number solutions following system eq...
16                     solve system equations x 5 -2x 2 
105       write expression match phrase '' q times 2 `` 
26         solve following system substitution 5x 7x 16 
Name: problem, Length: 200, dtype: object

In [76]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
vect_X_train = vectorizer.fit_transform(X_train)
feature_names =  vectorizer.get_feature_names_out()

In [77]:
len(feature_names)

2354

In [78]:
vect_X_test = vectorizer.transform(X_test)

In [79]:
cls = MultinomialNB()
cls.fit(vect_X_train, y_train)

MultinomialNB()

In [80]:
y_pred = cls.predict(vect_X_train)

In [81]:
np.unique(y_pred)

array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 19,
       20, 21, 22, 23, 24], dtype=int64)

In [82]:
accuracy_score(y_train, y_pred)*100

88.5

In [83]:
test_y_pred = cls.predict(vect_X_test)

In [84]:
accuracy_score(y_test, test_y_pred)*100

42.0

In [85]:
rdmfst = RandomForestClassifier(max_depth=20)
rdmfst.fit(vect_X_train, y_train)

RandomForestClassifier(max_depth=20)

In [86]:
y_pred = rdmfst.predict(vect_X_train)

In [87]:
accuracy_score(y_train, y_pred)*100

98.0

In [88]:
test_y_pred = rdmfst.predict(vect_X_test)

In [89]:
accuracy_score(y_test, test_y_pred)*100

54.0

In [90]:
dtree = DecisionTreeClassifier(max_depth=20)
dtree.fit(vect_X_train, y_train)

DecisionTreeClassifier(max_depth=20)

In [91]:
y_pred = dtree.predict(vect_X_train)

In [92]:
accuracy_score(y_train, y_pred)*100

93.0

In [93]:
test_y_pred = dtree.predict(vect_X_test)
accuracy_score(y_test, test_y_pred)*100

57.99999999999999

In [94]:
neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
neigh.fit(vect_X_train, y_train)

KNeighborsClassifier(weights='distance')

In [96]:
y_pred = neigh.predict(vect_X_train)

In [97]:
accuracy_score(y_train, y_pred)*100

98.0

In [98]:
test_y_pred = neigh.predict(vect_X_test)

In [99]:
accuracy_score(y_test, test_y_pred)*100

52.0

In [100]:
gboost = GradientBoostingClassifier(max_depth=3)
gboost.fit(vect_X_train, y_train)

GradientBoostingClassifier()

In [101]:
y_pred = gboost.predict(vect_X_train)
f'The accuracy score is: {round(accuracy_score(y_train, y_pred)*100, 2)}%'

'The accuracy score is: 98.0%'

In [102]:
test_y_pred = gboost.predict(vect_X_test)

In [103]:
f'The accuracy score is: {round(accuracy_score(y_test, test_y_pred)*100, 2)}%'

'The accuracy score is: 66.0%'

In [104]:
xgb = XGBClassifier()

In [105]:
xgb.fit(vect_X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [106]:
y_pred = xgb.predict(vect_X_train)
f'The accuracy score is: {round(accuracy_score(y_train, y_pred)*100, 2)}%'

'The accuracy score is: 97.5%'

In [107]:
test_y_pred = xgb.predict(vect_X_test)
f'The accuracy score is: {round(accuracy_score(y_test, test_y_pred)*100, 2)}%'

'The accuracy score is: 46.0%'

In [14]:
# Completely new approach using fasttext
csv_file = r'C:\Users\tanma\Downloads\train.csv'

with open(csv_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = list(reader)

fasttext_data = []
for row in data:
    text = row[0]
    text = remove_html_tags(text)
    label = row[1]
    fasttext_data.append("__label__{} {}".format(label, text))
    
text_file = 'train.txt'
with open(text_file, 'w', encoding='utf-8') as file:
    file.write('\n'.join(fasttext_data))


C:\Users\tanma\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [97]:
import random

In [48]:
model1 = fasttext.train_supervised(input=text_file, lr=0.5, epoch=100, wordNgrams=3,loss = 'softmax',bucket = 200000)

In [53]:
model2 = fasttext.train_supervised(input=text_file, lr=0.3, epoch=200, wordNgrams=1,loss = 'softmax',bucket = 200000)

In [60]:
model3 = fasttext.train_supervised(input=text_file, lr=0.7, epoch=100, wordNgrams=3,loss = 'ova',bucket = 200000)

In [70]:
model4 = fasttext.train_supervised(input=text_file, lr=0.2, epoch=500, wordNgrams=1,loss = 'ova',bucket = 200000)

In [71]:
model4.predict("<p>Pins Bowling Alley charges `$3.50` per game and `$4` to rent shoes. Strike Bowling Alley charges `$4` per game and `$2.50` to rent shoes. </p><p>Write two equations in slope-intercept form to represent the cost")

(('__label__16',), array([0.99814242]))

In [72]:
predictions = []
for i in range(len(df_test)):
    text = remove_html_tags(df_test['problem'].loc[i])
    prediction = model3.predict(text)[0][0]
    prediction = prediction.replace("__label__",'')
    predictions.append(prediction)

In [73]:
predictions

['19',
 '8',
 '8',
 '7',
 '5',
 '23',
 '3',
 '4',
 '24',
 '8',
 '4',
 '4',
 '2',
 '9',
 '11',
 '23',
 '2',
 '2',
 '4',
 '16',
 '16',
 '22',
 '9',
 '17',
 '10',
 '9',
 '16',
 '12',
 '13',
 '18',
 '14',
 '7',
 '7',
 '22',
 '11',
 '21',
 '19',
 '8',
 '18',
 '1',
 '17',
 '17',
 '15',
 '3',
 '12',
 '20',
 '0',
 '2',
 '0',
 '8',
 '23',
 '11',
 '24',
 '24',
 '15',
 '17',
 '20',
 '1',
 '13',
 '16',
 '21',
 '16',
 '9',
 '14',
 '1',
 '6',
 '13',
 '17',
 '18',
 '5',
 '4',
 '22',
 '8',
 '13',
 '9',
 '5',
 '0',
 '14',
 '1',
 '13',
 '7',
 '15',
 '16',
 '24',
 '11',
 '15',
 '6',
 '17',
 '9',
 '12',
 '8',
 '22',
 '24',
 '3',
 '14',
 '11',
 '20',
 '5',
 '23',
 '15',
 '6',
 '17',
 '14',
 '4',
 '3',
 '20',
 '5',
 '6',
 '3',
 '12',
 '0',
 '0',
 '9',
 '7',
 '12',
 '21',
 '23',
 '7',
 '19',
 '7',
 '8',
 '6',
 '20',
 '22',
 '2']

In [74]:
df_test['category'] = predictions

In [75]:
df_test

,problem,category
0,"<p>For graduating from middle school, Wayne r...",19
1,<p>Mrs. O'Hare bought `18` packs of pencils to...,8
2,<p>Annalise bought `p` pens last week and `15`...,8
3,<p>Felicia is decorating cupcakes for a party....,7
4,<p>Write an expression to match the phrase:</p...,5
...,...,...
120,<p>Veronica has `d` more dresses than Larisa d...,8
121,<p>Write an equation to match the phrase:</p><...,6
122,<p>Follow the directions below to create an eq...,20
123,<p>What is `7.5 ÷ 10`?</p><selectivedisplay da...,22


In [76]:
submission = 'submission.csv'
df_test.to_csv(submission,index = False)